# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [2]:
import pandas as pd

# Load dataset
df = pd.read_csv("tic-tac-toe.csv")

# Preview data
df.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True


In [3]:
# have a look at the structure
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [4]:
# check the distribution
df['class'].value_counts()


class
True     626
False    332
Name: count, dtype: int64

In [5]:
# is the Board itself good?
df.iloc[:5, :9]


,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,x,x,x,x,o,o,x,o,o
1,x,x,x,x,o,o,o,x,o
2,x,x,x,x,o,o,o,o,x
3,x,x,x,x,o,o,o,b,b
4,x,x,x,x,o,o,b,o,b


MY SANITY CHECK!

* Exactly 9 board-position columns

* Values limited to {x, o, b}

* No missing values

* Well balanced enough target distribution for classification

* Board encoding is consistent with Tic Tac Toe rules

✔ Dataset is reliable and clean!

In [6]:
encoding_map = {'b': 0, 'x': 1, 'o': -1}

for col in df.columns[:-1]:  # exclude target
    df[col] = df[col].map(encoding_map)

# Convert target to binary
df['class'] = df['class'].astype(int)

df.head()


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,-1,-1,1,-1,-1,1
1,1,1,1,1,-1,-1,-1,1,-1,1
2,1,1,1,1,-1,-1,-1,-1,1,1
3,1,1,1,1,-1,-1,-1,0,0,1
4,1,1,1,1,-1,-1,0,-1,0,1


✔ This encoding preserves the semantics of the game AND Avoids unnecessary one-hot expansion (non-numeric variables into binary)

In [7]:
X = df.iloc[:, :-1].values   # 9 board positions
y = df.iloc[:, -1].values   # X wins (1) or not (0)

print(X.shape, y.shape)


(958, 9) (958,)


In [8]:
# Even though values are small, normalization stabilizes gradient descent and improves convergence
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [9]:
# Split Training and Test Data - 20% of the data for unbiased evaluation should be reserved.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_test.shape)


(766, 9) (192, 9)


In [10]:
import sys
print(sys.version)


3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [11]:
import sys
import tensorflow as tf

print(sys.version)
print(tf.__version__)


3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
2.20.0


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(X_train.shape, X_test.shape)


(766, 9) (192, 9)


In [17]:
# create a sequential model, fully connected (Dense) neural network, appropriate for structured/tabular data.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [18]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(9,)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(2, activation='softmax')  # Binary classification with probability output
])


c:\Users\profe\Documents\Ironhack\lab-neural-networks\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# compile the model!
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [20]:
# fit the training data
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6340 - loss: 0.6505 - val_accuracy: 0.6364 - val_loss: 0.6326
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6601 - loss: 0.5889 - val_accuracy: 0.6429 - val_loss: 0.5987
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7010 - loss: 0.5504 - val_accuracy: 0.6558 - val_loss: 0.5662
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7271 - loss: 0.5187 - val_accuracy: 0.7078 - val_loss: 0.5382
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7958 - loss: 0.4850 - val_accuracy: 0.7597 - val_loss: 0.5081
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8382 - loss: 0.4480 - val_accuracy: 0.7792 - val_loss: 0.4643
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8578 - loss: 0.4096 - val_accuracy: 0.8052 - val_loss: 0.4195
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9118 - loss: 0.3540 - val_accuracy: 0.8701 - val_loss

In [21]:
# let's evaluate on the test data - 50 epochs above is sufficient for convergence
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")


Test Accuracy: 0.9896
Test Loss: 0.0278


In [23]:
# save the model
model.save("tic-tac-toe.keras")


In [24]:
from tensorflow.keras.models import load_model
model = load_model("tic-tac-toe.keras")


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [26]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("tic-tac-toe.csv")

# Encode all features
X = df.iloc[:, :-1].copy()  # all columns except label
y = df.iloc[:, -1].copy()   # last column is the label

# Apply LabelEncoder to every column
le_X = {}  # to save encoders for future use
for col in X.columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    le_X[col] = le  # save encoder in case we need to decode later

# Encode labels
le_y = LabelEncoder()
y = le_y.fit_transform(y)


In [27]:
# split then scale!
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Load the saved model and make predictions
from tensorflow.keras.models import load_model
import numpy as np

# Load the model
model = load_model("tic-tac-toe.keras")

# Pick a few random test rows
num_samples = 5
random_idx = np.random.choice(len(X_test_scaled), num_samples, replace=False)
X_sample = X_test_scaled[random_idx]
y_sample = y_test[random_idx]

# Predict
pred_probs = model.predict(X_sample)
pred_classes = (pred_probs > 0.5).astype(int).flatten()

# Compare predictions with actual labels
for i in range(num_samples):
    print(f"Row {i}: Prediction={pred_classes[i]}, Actual={y_sample[i]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Row 0: Prediction=1, Actual=0
Row 1: Prediction=0, Actual=1
Row 2: Prediction=1, Actual=1
Row 3: Prediction=0, Actual=1
Row 4: Prediction=1, Actual=1


In [31]:
# Use the encoded sample rows (0/1) and decode them
X_sample_encoded = X_test[random_idx]  # numeric encoded values
X_sample_orig_decoded = []

for i in range(num_samples):
    row_decoded = []
    for j, col in enumerate(X.columns):
        # decode numeric value back to original 'x'/'o'
        value = le_X[col].inverse_transform([X_sample_encoded[i][j]])[0]
        row_decoded.append(value)
    X_sample_orig_decoded.append(row_decoded)


In [32]:
for i in range(num_samples):
    board = X_sample_orig_decoded[i]
    actual_label = le_y.inverse_transform([y_test[random_idx[i]]])[0]
    prediction = pred_labels[i]
    print(f"Board {i}: {board}")
    print(f"  Prediction: {prediction}, Actual: {actual_label}")
    print("-" * 30)


Board 0: ['x', 'x', 'x', 'b', 'b', 'o', 'o', 'x', 'o']
  Prediction: False, Actual: True
------------------------------
Board 1: ['b', 'x', 'b', 'o', 'o', 'o', 'x', 'b', 'x']
  Prediction: True, Actual: False
------------------------------
Board 2: ['x', 'o', 'b', 'o', 'b', 'o', 'x', 'x', 'x']
  Prediction: True, Actual: True
------------------------------
Board 3: ['x', 'x', 'x', 'b', 'b', 'b', 'o', 'b', 'o']
  Prediction: False, Actual: True
------------------------------
Board 4: ['o', 'x', 'x', 'x', 'o', 'o', 'o', 'x', 'x']
  Prediction: False, Actual: False
------------------------------


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 1️⃣ Load dataset
df = pd.read_csv("tic-tac-toe.csv")

# 2️⃣ Separate features and label
X = df.iloc[:, :-1].copy()
y = df.iloc[:, -1].copy()

# 3️⃣ Encode features (x/o -> 0/1)
le_X = {}
X_encoded = X.copy()
for col in X.columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X[col])
    le_X[col] = le

# 4️⃣ Encode labels
le_y = LabelEncoder()
y_encoded = le_y.fit_transform(y)  # 0/1

# 5️⃣ Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded.values, y_encoded, test_size=0.2, random_state=42
)

# 6️⃣ Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 7️⃣ Build the improved model
model = Sequential([
    Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'),  # more neurons
    Dense(16, activation='relu'),  # additional hidden layer
    Dense(8, activation='relu'),   # another hidden layer
    Dense(1, activation='sigmoid')  # output layer for binary classification
])

# 8️⃣ Compile model with custom learning rate
learning_rate = 0.01  # try adjusting this
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 9️⃣ Fit model with more epochs
epochs = 50  # can increase if accuracy is still low
history = model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=8, verbose=1,
                    validation_split=0.1)

# 1️⃣0️⃣ Evaluate performance on test set
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# 1️⃣1️⃣ Save the improved model
model.save("tic-tac-toe-improved.keras")

# ✅ Optional: make predictions on a few test rows
num_samples = 5
random_idx = np.random.choice(len(X_test_scaled), num_samples, replace=False)
X_sample_scaled = X_test_scaled[random_idx]
y_sample = y_test[random_idx]

pred_probs = model.predict(X_sample_scaled)
pred_classes = (pred_probs > 0.5).astype(int).flatten()
pred_labels = le_y.inverse_transform(pred_classes)

for i in range(num_samples):
    board_encoded = X_test[random_idx][i]
    board_decoded = [le_X[col].inverse_transform([board_encoded[j]])[0] for j, col in enumerate(X.columns)]
    actual_label = le_y.inverse_transform([y_sample[i]])[0]
    print(f"Board {i}: {board_decoded}")
    print(f"  Prediction: {pred_labels[i]}, Actual: {actual_label}")
    print("-" * 30)


Epoch 1/50


c:\Users\profe\Documents\Ironhack\lab-neural-networks\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6459 - loss: 0.6231 - val_accuracy: 0.7273 - val_loss: 0.5652
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7344 - loss: 0.5391 - val_accuracy: 0.8312 - val_loss: 0.4458
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7808 - loss: 0.4781 - val_accuracy: 0.8182 - val_loss: 0.4639
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8302 - loss: 0.4147 - val_accuracy: 0.8312 - val_loss: 0.4157
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8476 - loss: 0.3754 - val_accuracy: 0.8571 - val_loss: 0.3762
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8650 - loss: 0.3272 - val_accuracy: 0.8442 - val_loss: 0.3543
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8839 - loss: 0.2855 - val_accuracy: 0.9091 - val_loss: 0.2553
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9057 - loss: 0.2279 - val_accuracy: 0.8442 - val_loss: 0.2848
Epo

**Which approach(es) did you find helpful to improve your model performance?**

✅ Hmmmmm - summary of what really helped

For this challenge, the most impactful improvements were:

Adding more layers/neurons → increased model capacity.

Adjusting learning rate → faster and more stable convergence.

Training for more epochs → allowed the model to reach low loss and high accuracy